# Краткий отчет по COVID-19
***

## Адрес для обновления данных

- Данные принадлежат Университету Джона Хопкинса и получены на [Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset#time_series_covid_19_confirmed.csv).

# 1 Загрузим библиотеки
***

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 

from matplotlib import pyplot as plt

import plotly as py
import plotly.express as px
import plotly.graph_objs as go

from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## 1.1 Объявим класс для исследования и наполним его методами

In [ ]:
class DataExplorer:
      
    def histogram(self, data, n_bins, range_start, range_end, grid, cumulative=False, x_label = '', y_label = '', title = ''):
        
        """
        Простая гистограмма
        
        Пример:
        histogram(df, 100, 0, 150, True, 'Количество иксов', 'Количество игриков', 'Заголовок')
        
        data - датасет
        n_bins - количество корзин
        range_start - минимальный икс для корзины
        range_end - максимальный икс для корзины
        grid - рисовать сетку или нет (False / True)
        
        
        histogram(data, n_bins, range_start, range_end, grid, x_label = "", y_label = "", title = "")
        """
        
        # Создаем объект - график
        _, ax = plt.subplots()

        # Задаем параметры
        ax.hist(data, bins = n_bins, range = (range_start, range_end), cumulative = cumulative, color = '#4169E1')

        # Добавляем сетку
        if grid == True:
            ax.grid(color='grey', linestyle='-', linewidth=0.5)
        else:
            pass

        # Добавляем медиану, среднее и квартили
        ax.axvline(data.median(),linestyle = '--', color = '#FF1493', label = 'median')
        ax.axvline(data.mean(),linestyle = '--', color = 'orange', label = 'mean')
        ax.axvline(data.quantile(0.1),linestyle = '--', color = 'yellow', label = '1%')
        ax.axvline(data.quantile(0.99),linestyle = '--', color = 'yellow', label = '99%')
        ax.legend()
        ax.set_ylabel(y_label)
        ax.set_xlabel(x_label)
        ax.set_title(title)        
        

    def plotly_bar(self, data, x, y, color, x_axis_title, y_axis_title, title):
        """
        Рисует интерактивный столбчатый график
        
        Пример:
        explorer.plotly_bar(top_ten, 'city', 'average_flights', 'average_flights', '', 'Количество рейсов', 'Топ-10 городов по количеству рейсов')
        """
        
        fig = px.bar(data, x=data[x], y=data[y], color=color, color_continuous_scale='portland')
        
        fig.update_layout(title=title,
        xaxis_title=x_axis_title,
        yaxis_title=y_axis_title,
        font=dict(
        family="Garamond', monospace",
        size=13,
        color='#7f7f7f'),
        template='plotly_white',
        showlegend=False)
        
        fig.show() 
        
        
    def plotly_choropleth(self, data, z='confirmed', title=''):
        """
        Рисует интерактивную карту
        
        Пример:
        explorer.plotly_choropleth(z='recovered', title='Число вылечившхся на 19.01.2021 года')
        """
            
        fig = go.Figure(data=go.Choropleth(
            locations = data['country'],
            locationmode = 'country names',
            z = data[z],
            colorscale = 'Redor',
            marker_line_color = 'black',
            marker_line_width = 0.5,
        ))
        
        fig.update_layout(title_text = title,
                          title_x = 0.5,
                          geo=dict(showframe = False, showcoastlines = False, projection_type = 'equirectangular'))
        
        fig.show()
        
        
    def plotly_animated_choropleth(self, data, color='confirmed', title=''):        
        fig = px.choropleth(data, 
                    locations='country', 
                    locationmode = 'country names',
                    color=color, 
                    hover_name='country', 
                    animation_frame='date'
                   )
        fig.update_layout(
            title_text = title,
            title_x = 0.5,
            geo=dict(
                showframe = False,
                showcoastlines = False,
            ))

        fig.show()
        
    def plotly_sided_bar(self, data):
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=data['country'],
            y=data['confirmed'],
            name='confirmed',
            marker_color='blue'
        ))
        fig.add_trace(go.Bar(
            x=data['country'],
            y=data['recovered'],
            name='recovered',
            marker_color='red'))
        fig.add_trace(go.Bar(
            x=data['country'],
            y=data['deaths'],
            name='deaths',
            marker_color='green'
        ))
        fig.update_layout(font=dict(family="Garamond', monospace",
                                    size=13,
                                    color='#7f7f7f'),
                          template='plotly_white',
                          xaxis_tickangle=-45,
                          showlegend=True)
        fig.show()
        
    def plotly_stack(self, data):    
        fig = go.Figure(data=[
            go.Bar(name='confirmed', x=data['country'], y=data['confirmed']),
            go.Bar(name='recovered', x=data['country'], y=data['recovered']),
            go.Bar(name='deaths', x=data['country'], y=data['deaths'])
        ])

        fig.update_layout(barmode='stack',
                          font=dict(family="Garamond', monospace",
                                    size=13,
                                    color='#7f7f7f'),
                          template='plotly_white',
                          xaxis_tickangle=-45,
                          showlegend=True)
        fig.show()
        

    def firstsight(self, data):
        
        """
        Возврашает пять первых, последних и случайных элементов датасета для дальнейшего вывода с помощью Display
        
        Пример:
        head, tail, sample = explorer.firstsight(df)
        """
        
        head = data.head(5)
        tail = data.tail(5)
        sample = data.sample(5)
        
        return head, tail, sample
    
    def reporter(self, confirmed, deaths, recovered):
        cfr = (deaths / (deaths + recovered)) * 100

        print('1. По состоянию на 19.01.2021 года:' + '\n' + 
              ' - заболело {} '.format(confirmed)  + 'человек' + '\n' +
              ' - умерло {} '.format(deaths)  + 'человек' + '\n' +
              ' - выздоровело {} '.format(recovered)  + 'человек' + '\n' +
              '2. На сегодняшний день, исходя из представленных данных,' +
              'принимая во внимание короткий период наблюдений летальность COVID-19 ' +
              'составляет {:.2f} %'.format(cfr))    
        
    class Display(object):
        """
        Выводит HTML представление нескольких объектов
        
        Пример:
        Display(head, tail, sample)
        
        """
        template = """<div style="float: left; padding: 10px;">
        <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
        </div>"""
        def __init__(self, *args):
            self.args = args

        def _repr_html_(self):
            return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                             for a in self.args)

        def __repr__(self):
            return '\n\n'.join(a + '\n' + repr(eval(a))
                               for a in self.args)

In [ ]:
explorer = DataExplorer()

# 2 Прочитаем данные

#### - прочитаем данные в переменную

In [ ]:
df = pd.read_csv('covid_19_data.csv')

# 3 Изменим регистр в наименовании столбцов, перименуем столбцы, приведенм данные к целочисленному типу

#### - приведем к строчному написанию заголовки столбцов

In [ ]:
df.columns = map(str.lower, df.columns)

#### - переименуем столбцы с датой, регионом и страной, отбросим столбец 'last update' и 'sno' как неинформативные

In [ ]:
df.drop(['sno','last update'], axis=1, inplace=True)

In [ ]:
df.rename(columns={'observationdate': 'date',
                   'province/state': 'state',
                   'country/region':'country'}, inplace=True)

#### - приведем столбы с наблюдениями о числе зараженных, умерших и выздоровевших в целочисленный тип, поскольку это дискретные величины

In [ ]:
df['confirmed'] = df.confirmed.astype('int')
df['deaths'] = df.deaths.astype('int')
df['recovered'] = df.recovered.astype('int')

# 4 Подсчитаем летальность от COVID-19 на 19.01.2021 года

#### - определение летальности

- **[Летальность](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D1%82%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C)** — показатель медицинской статистики, равный отношению числа умерших от определённого заболевания или иного нарушения здоровья за определённый период времени к общему числу людей, имевших тот же диагноз в рамках того же периода времени. Обычно выражается в процентах. Показатель летальности не постоянен, он может изменяться между популяциями, а также с течением времени. Летальность зависит от разных факторов, которые могут включать особенности патогена в случае заболевания, состояние организма, внешние обстоятельства, способы лечения и качество ухода за больными. Расчётным периодом часто является год.

#### - формула для расчета

$$
\displaystyle {{умерло}\over{(умерло + вылечилось)}} \times 100 %
$$

#### - подготовим данные для расчета

- Сгруппируем по странам и датам, отсортируем по дате и избавимся от дубликатов, чтобы оставить только последние заиксированные наблюдения, отберем только наблюдения с потвержденными случаями

In [ ]:
df_total = df.groupby(['country', 'date']).sum().reset_index().sort_values('date', ascending=False).drop_duplicates(subset = ['country'])
df_total = df_total[df_total['confirmed'] > 0]

#### - соберем данные о зараженных, умерших и выздоровевших

In [ ]:
confirmed = df_total.confirmed.sum()
deaths = df_total.deaths.sum()
recovered = df_total.recovered.sum()

#### - расчитаем летальность по формуле

In [ ]:
cfr = (df_total.deaths.sum() / (df_total.deaths.sum() + df_total.recovered.sum())) * 100

&#9889; **Вывод:** 

In [ ]:
explorer.reporter(confirmed, deaths, recovered)

# 5 Визуализируем данные о числе зараженных на 19.01.2021 года

#### - визуализируем общую карту с числом подтвержденных случаев

In [ ]:
explorer.plotly_choropleth(df_total, z='confirmed', title='Подтвержденные случаи на 19.01.2021 года')

#### - визуализируем соотношение заболевших, умерших и выздоровевших для первых десяти странами по числу выявленных зараженных

In [ ]:
top_ten = df_total.sort_values(by='confirmed', ascending=False).head(10)

In [ ]:
explorer.plotly_sided_bar(top_ten)

#### - выведем три страны с самым большим числом заболевших на сегодняшний день

In [ ]:
top_ten.head(3)

#### - выведем статистику для России на сегодняшний день

In [ ]:
df_total[df_total['country'] == 'Russia']